# The SVM vs. Logistic Regression Showdown

In this lab, you will practice working with non-linear kernels combined with logistic regression and SVM classifiers. The goal is to compare these commonly used techniques. Which comes out on top in terms of accuracy? Runtime? Is there much of a difference at all? Is the dominance of the SVM classifier in machine learning pedagogy justified?

## Loading the Data

First, we load all the packages we'll need.

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import pairwise_kernels
import scipy
from sklearn import svm, linear_model
from sklearn.model_selection import GridSearchCV
import time

Again we download the data from the Tensorflow package, which you will need to install.  You can get the data from other sources as well.

In the Tensorflow dataset, the training and test data are represented as arrays:

     Xtr.shape = 60000 x 28 x 28
     Xts.shape = 10000 x 28 x 28
     
The test data consists of `60000` images of size `28 x 28` pixels; the test data consists of `10000` images.

In [2]:
import tensorflow as tf

(Xtr_raw,ytr),(Xts_raw,yts) = tf.keras.datasets.mnist.load_data()

print('Xtr shape: %s' % str(Xtr_raw.shape))
print('Xts shape: %s' % str(Xts_raw.shape))

ntr = Xtr_raw.shape[0]
nts = Xts_raw.shape[0]
nrow = Xtr_raw.shape[1]
ncol = Xtr_raw.shape[2]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Xtr shape: (60000, 28, 28)
Xts shape: (10000, 28, 28)


Each pixel value is from `[0,255]`.  For this lab, it will be convenient to recale the value to -1 to 1 and reshape it as a `ntr x npix` and `nts x npix`.

In [3]:
npix = nrow*ncol
Xtr = Xtr_raw.reshape((ntr,npix))
print(Xtr[1,:])
Xtr = (Xtr/255 - .5)
print(Xtr[1,:])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  51 159 253 159  50   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  48 238 252 252 252 237   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  54 227 253 252 239 233 252  57   6   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  10  60 224 252 253 252 202  84 252
 253 122   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 163 252 252 252 253 252 252  96 189 253 167   

In [4]:
npix = nrow*ncol
Xtr = (Xtr_raw/255 - 0.5)
Xtr = Xtr.reshape((ntr,npix))

Xts = (Xts_raw/255 - 0.5)
Xts = Xts.reshape((nts,npix))

In [5]:
print(Xtr)

[[-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 ...
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]]


For this lab we're only going to use a fraction of the MNIST data -- otherwise our models will take too much time and memory to run. Using only part of the training data will of course lead to worse results. Given enough computational resources and time, we would ideally be running on the full data set. The follow code creates a new test and train set, with 10000 examples for train and 5000 for test.

In [6]:
ntr1 = 10000
nts1 = 5000
Iperm = np.random.permutation(ntr1)
Xtr1 = Xtr[Iperm[:ntr1],:]
ytr1 = ytr[Iperm[:ntr1]]
Iperm = np.random.permutation(nts1)
Xts1 = Xts[Iperm[:nts1],:]
yts1 = yts[Iperm[:nts1]]

## Problem set up and establishing a baseline

To simplify the problem (and speed things up) we're also going to restrict to binary classification. In particular, let's try to design classifier a that separates the 8's from all other digits.

Create binary 0/1 label vectors `ytr8` and `yts8` which are 1 wherever `ytr1` and `yts1` equal 8, and 0 everywhere else.

In [ ]:
ytr8 = (ytr1 == 8).astype(int)
yts8 = (yts1 == 8).astype(int)
print(ytr8)
print(yts8)

[0 1 0 ... 1 0 0]
[0 0 0 ... 0 0 1]


Most of the digits in the test dataset aren't equal to 8. So if we simply guess 0 for every image in `Xts`, we might expect to get classification accuracy around 90%. Our goal should be to significantly beat this **baseline**.

Formally, write a few lines of code to check what test error would be achieved by the all zeros classifier.

In [ ]:
all_zeros_acc = np.mean(yts8 == 0)
print('All Zeros Accuracy = {0:.2f}%'.format(all_zeros_acc * 100))


All Zeros Accuracy = 90.22%


As a second baseline, let's see how we do with standard (non-kernel) logistic regression. As in the MNIST demo, you can use `scikit-learn`'s built in function `linear_model.LogisticRegression` to fit the model and compute the accuracy. Use no regularization and the `lbfgs` solver. You should acheive an improvement to around 93-95%.

In [25]:
lr = linear_model.LogisticRegression(solver='lbfgs', C=1e10, max_iter=1000)
lr.fit(Xtr1, ytr8)
lr_preds = lr.predict(Xts1)
lr_acc = np.mean(lr_preds == yts8)
print('LR Accuracy = {0:.2f}%'.format(lr_acc * 100))


LR Accuracy = 93.60%


## Kernel Logistic Regression

To improve on this baseline performance, let's try using the logistic regression classifier with a *non-linear* kernel. Recall from class that any non-linear kernel similarity function $k(\vec{w},\vec{z})$ is equal to $\phi(\vec{w})^T\phi(\vec{z})$ for some feature transformation $\phi$. However, we typically do not need to compute this feature tranformation explicitly: instead we can work directly with the kernel gram matrix $K \in \mathbb{R}^{n\times n}$. Recall that $K_{i,j} = k(\vec{x}_i,\vec{x}_j)$ where $\vec{x}_i$ is the $i^\text{th}$ training data point.

For this lab we will be using the radial basis function kernel. For a given scaling factor $\gamma$ this kernel is defined as:
$$
k(\vec{w},\vec{z}) = e^{-\gamma\|\vec{w}-\vec{z}\|_2^2}
$$

In [ ]:
def rbf_kernel(w,z,gamma):
    d = w - z
    return np.exp(-gamma*np.sum(d*d))

Construct the kernel matrix `K1` for `Xtr1` with `gamma = .05`.

In [ ]:
gamma = .05
K1 = np.zeros((ntr1, ntr1))
for i in range(ntr1):
    for j in range(ntr1):
        K1[i, j] = rbf_kernel(Xtr1[i], Xtr1[j], gamma)

If you used a for loop (which is fine) your code might take several minutes to run! Part of the issue is that Python won't know to properly parallize your for loop. For this reason, when constructing kernel matrices it is often faster to us a built-in, carefully optimized function with explicit parallelization. Scikit learn provides such a function through their `metrics` library.

Referring to the documentation here
https://scikit-learn.org/stable/modules/metrics.html#metrics, use this built in function to recreate the same kernel matrix you did above. Store the result at `K`.

In [28]:
K = pairwise_kernels(Xtr1, Xtr1, metric='rbf', gamma=gamma)

Check that you used the function correctly by writing code to confirm that `K = K1`, or at least that the two are equal up to very small differences (which could arise due to numerical precision issues). Try to do this **without a for loop** for full credit. You will get partial credit if you use a for loop.

In [29]:
difference = np.max(np.abs(K - K1))
print(f"Maximum difference between matrices: {difference}")

difference2 = np.allclose(K, K1)
print(f"Matrices are effectively equal: {difference2}")

Maximum difference between matrices: 1.765254609153999e-14
Matrices are effectively equal: True


When using a non-linear kernel, it is important to check that you have chosen reasonable parameters (in our case the only parameter is `gamma`). We typically do not want $k(\vec{x}_i,\vec{x}_j)$ to be either negligably small, or very large for all $i\neq j$ in our data set, or we won't be able to learn anything. For the RBF kernel this means that, for any $\vec{x}_i$ we don't want $k(\vec{x}_i,\vec{x}_j)$ very close to 1 (e.g. .9999) for all $j$, or very close to $0$ (e.g. 1e-8) for all $j$.

Let's just check that we're in good shape for the first data vector $\vec{x}_0$. Do so by printing out the 10 largest and 10 smallest values of $k(\vec{x}_0,\vec{x}_j)$ for $j\neq 0$. Note that we always have $k(\vec{x}_0,\vec{x}_0) = 1$ for the RBF kernel.

In [ ]:
similarities = K[0, 1:]
sorted_similarities = np.sort(similarities)

print('Maximum similarities: \n' + str(sorted_similarities[-10:]))
print('Minimum similarities: \n' + str(sorted_similarities[:10]))

Maximum similarities: 
[0.07580635 0.08055649 0.08568271 0.08904923 0.10209036 0.10698518
 0.11096823 0.16795797 0.18964078 0.22482328]
Minimum similarities: 
[5.72009736e-05 1.14743427e-04 1.19927539e-04 1.23629394e-04
 1.28781213e-04 1.28974951e-04 1.30022886e-04 1.54589014e-04
 1.67103086e-04 1.69195844e-04]


### Implementation
Maybe surprisingly Scikit learn does not have an implementation for kernel logistic regression. So we have to implement our own!

Write a function function `log_fit` that minimizes the $\ell_2$-regularized logisitic regression loss:
$$
L(\boldsymbol{\alpha}) =\sum_{i=1}^n (1-y_i)(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\vec{\alpha}) - \log(h(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\boldsymbol{\alpha}) + \lambda \|\phi(\mathbf{X})^T\boldsymbol{\alpha}\|_2^2.
$$
As input it takes an $n\times n$ kernel matrix $K$ for the training data, an $n$ length vector `y` of binary class labels, and regularization parameter `lamb`.

To implement your function you can either use your own implmentation of gradient descent or used a built in minimizer from `scipy.optimize.minimize`. I recommend the later approach: as we saw in the last lab, gradient descent can converge slowly for this objective.

In [ ]:
def loss_function(alpha, K, y, lamb):
    # Compute the loss function for the SVM

    # Decision scores
    scores = K @ alpha

    # log(1/(1+exp(-z))) = -log(1+exp(-z))
    # For positive z: -log(1+exp(-z))
    # For negative z: -z-log(1+exp(z))
    pos_idx = scores >= 0
    neg_idx = ~pos_idx

    # Setup log_term
    log_term = np.zeros_like(scores)

    # Compute log_term for positive and negative scores
    log_term[pos_idx] = np.log(1 + np.exp(-scores[pos_idx]))
    log_term[neg_idx] = -scores[neg_idx] + np.log(1 + np.exp(scores[neg_idx]))

    # First term: contribution from negative examples
    # Second term: contribution from positive examples
    # Third term: regularization
    return np.sum((1-y) * scores + log_term) + lamb * (alpha.T @ K @ alpha)

In [36]:
def gradient_function(alpha, K, y, lamb):
    # Compute decision scores
    scores = K @ alpha
    # Compute predicted probabilities using sigmoid
    probs = 1 / (1 + np.exp(-scores))
    # Compute gradient
    grad = K @ (probs - y) + 2 * lamb * (K @ alpha)
    return grad


In [40]:
def log_fit(K,y,lamb,nit=1000):
    n = K.shape[0]
    zeroes = np.zeros(n)

    # Gradient descent
    result = scipy.optimize.minimize(
        fun=loss_function,
        x0=zeroes,
        args=(K, y, lamb),
        jac=gradient_function,
        method='L-BFGS-B',
        options={'maxiter': nit}
    )

    return result.x

Use the `log_fit` function defined above to find parameters `alpha` for the kernel logistic regression model using `lamb = 0` and `K` as constructed above (with `gamma = .05`).

Suppose we have a test dataset with $m$ examples $\vec{w}_1,\ldots, \vec{w}_m$. Once we obtain a coefficient vector $\alpha$, making predictions for any $\vec{w}_j$ in the test set requires computing:
$$
{y}_{j} = \sum_{i=1}^n \alpha_i \cdot k(\vec{w}_{j}, \vec{x}_i).
$$
where $\vec{x}_1, \ldots \vec{x}_n$ are our training data vectors. We classify $\vec{w}_{j}$ in class 0 if ${y}_{j} \leq 0$ and in class 1 if ${y}_{j} > 0$.

This computation can be rewritten in matrix form as follows:
$$
\vec{y}_{test} = K_{test}\vec{\alpha},
$$
where $\vec{y}_{text}$ is an $m$ length vector and $K_{test}$ is a $m\times n$ matrix whose $(j,i)$ entry is equal to $k(\vec{w}_{j}, \vec{x}_i)$. We classify $\vec{w}_{j}$ in class 0 if $\vec{y}_{test}[j] \leq 0$ and in class 1 if $\vec{y}_{test}[j] > 0$.


Use the `pairwise_kernels` function to construct $K_{test}$. Then make predictions for the test set and evaluate the accuracy of our kernel logistic regression classifier. You should see a pretty substantial lift in accuracy to around $97\%$

In [ ]:
alpha = log_fit(K, ytr8, lamb=0)

Ktest = pairwise_kernels(Xts1, Xtr1, metric='rbf', gamma=gamma)

y_score = Ktest @ alpha
yhat = (y_score > 0).astype(int)
acc = np.mean(yhat == yts8)

In [43]:
print("Test accuracy = %f" % acc)

Test accuracy = 0.979800


## Kernel Support Vector Machine

The goal of this lab is to compare Kernel Logistic Regression to Kernel Support Vector machines. Following `demo_mnist_svm.ipynb` create and train an SVM classifier on `Xtr1` and `ytr8` using an RBF kernel with `gamma = .05` (the same value we used for logistic regression aboe). Use margin parameter `C = 10`.

In [44]:
start_time = time.time()
svm_model = svm.SVC(kernel='rbf', gamma=gamma, C=10)
svm_model.fit(Xtr1, ytr8)

SVC(C=10, gamma=0.05)

Calculate and print the accuracy of the SVM classifier. You should obtain a similar result as for logistic regression: something close to $97\%$ accuracy.

In [46]:
ysvm = svm_model.predict(Xts1)
svm_acc = np.mean(ysvm == yts8)
print("Test accuracy = %f" % svm_acc)

Test accuracy = 0.980200


## The Showdown

Both SVM classifiers and kernel logisitic regression require tuning parameters to obtain the best possible result. In our setting we will stick with an RBF kernel (although this could be tuned). So we only consider tuning the kernel width parameter `gamma`, as well as the regularization parameter `lamb` for logistic regression, and the margin parameter `C` for SVM. We will choose parameters using for-loops and train-test cross validation.

Train a logistic regression classifier with **all combinations** of the parameters included below in vectors `gamma` and `lamb`. For each setting of parameters, compute and print:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

For computing runtime you might want to use the `time()` function from the `time` library, which we already imported ealier.

In [48]:
gamma = [.1, .05,.02,.01,.005]
lamb = [0,1e-6,1e-4,1e-2]

best_lr_acc = 0
best_lr_params = None
best_lr_time = float('inf')

for i in gamma:
    for j in lamb:
        start_time = time.time()
        
        # Create kernel matrix
        K = pairwise_kernels(Xtr1, Xtr1, metric='rbf', gamma=i)
        
        # Train model
        alpha = log_fit(K, ytr8, lamb=j)
        
        # Make predictions
        Ktest = pairwise_kernels(Xts1, Xtr1, metric='rbf', gamma=i)
        y_score = Ktest @ alpha
        yhat = (y_score > 0).astype(int)
        
        elapsed = time.time() - start_time
        
        acc = np.mean(yhat == yts8)
        if acc > best_lr_acc:
            best_lr_acc = acc
            best_lr_params = (i, j)
            best_lr_time = elapsed


/var/folders/4_/h0p5fc5x7jnbcxr3dl4b8mf00000gn/T/ipykernel_90075/48436916.py:5: RuntimeWarning: overflow encountered in exp
  probs = 1 / (1 + np.exp(-scores))


Now let's do the same thing for the kernel Support Vector Classifier. Train an SVM classifier with **all combinations** of the parameters included below in vectors `gamma` and `C`. For each setting of parameters, compute:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

In [49]:
gamma = [.1, .05,.02,.01,.005]
C = [.01,.1,1,10]

best_svm_acc = 0
best_svm_params = None
best_svm_time = float('inf')

for g in gamma:
    for c in C:
        start_time = time.time()
        
        # Train model
        svm_model = svm.SVC(kernel='rbf', gamma=g, C=c)
        svm_model.fit(Xtr1, ytr8)
        
        # Make predictions
        ysvm = svm_model.predict(Xts1)
        
        elapsed = time.time() - start_time
        
        acc = np.mean(ysvm == yts8)
        if acc > best_svm_acc:
            best_svm_acc = acc
            best_svm_params = (g, c)
            best_svm_time = elapsed


TODO: What was the best test error achieved, and what setting of parameters achieved this error? Was the kernel logistic regression classifier more sensitive to changes in `gamma` or `lamb`? Discuss in 1-3 short sentences below.

TODO: What was the best test error achieved, and what setting of parameters achieved this error? Which performed better in terms of accuracy, the SVM or logisitic regression classifier? How about in terms of runtime?

In [50]:
print("\nBest logistic regression result:")
print(f"gamma={best_lr_params[0]}, lambda={best_lr_params[1]}, accuracy={best_lr_acc:.4f}, time={best_lr_time:.2f}s")

print("\nBest SVM result:")
print(f"gamma={best_svm_params[0]}, C={best_svm_params[1]}, accuracy={best_svm_acc:.4f}, time={best_svm_time:.2f}s")


Best logistic regression result:
gamma=0.02, lambda=1e-06, accuracy=0.9854, time=63.10s

Best SVM result:
gamma=0.01, C=10, accuracy=0.9870, time=3.07s


Best accruacy for the lr was 95.54%.

klr was more sensitive to the gamma parameter since it controls width and complexity.

It ended up taking what felt like eons to complete due to these high gamma values. 

Funnily enough, the SVM classifier had a higher accuracy and took much less time. 

This is likely due to the optimization of the scilearn library.

We did make the nit 1000 for lr which could have lead to redundant calcs, but I think this might be unlikely since we used a library...

**NOTE:** For `sklearns`'s built in classifiers, including svm.SVC, there is a function called `GridSearchCV` which can automatically perform hyperparamater tuning for you. The main advantage of the method (as opposed to writing for-loops) is that it supports parallelization, so it can fit with different parameters at the same time. The function also supports automatic $k$-fold cross-validation (instead of simple train/test split).

You might be interested in using this function in the future. If so, please check out the tutorial in the following lab from previous year: https://github.com/sdrangan/introml/blob/master/unit08_svm/lab_emnist_partial.ipynb.